In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import sys
import datetime

In [2]:

loader = TextLoader("llama2_text.txt")
documents=loader.load()

In [3]:
text_splitter=RecursiveCharacterTextSplitter(
                                             chunk_size=500,
                                             chunk_overlap=20)
text_chunks=text_splitter.split_documents(documents)

print(text_chunks)

[Document(page_content='Please complete this survey.\nSurvey!\nPRIMARY APPLICANT DETAILS\nAppointment(s) Made By:\nAjay Kumar\nNumber of Applicants:\n1\nPassport Number:\nT2849736\nVisa Class:\nL-1 (Blanket)\nVisa Category:\nL-1 (Blanket)\nVisa Priority:\nEnglish\nDS-160 Confirmation Number:\nAA00C9ZLS1\nOFC APPOINTMENT DETAILS\nNumber of OFC Appointments:\n1\nOFC Appointment Number:\n1\nApplicant Name:\nAjay Kumar\nEmbassy/Consulate/OFC:\nCHENNAI VAC\nStreet Address:\nNo 82, Kodambakkam High Road,\nStreet Address Cont.:\nNungambakkam,', metadata={'source': 'llama2_text.txt'}), Document(page_content='Nungambakkam,\nCity, Postal Code:\nChennai, 600034\nOFC Appointment Date:\n8/30/2023 9:30:00 AM\nCONSULAR APPOINTMENT DETAILS\nNumber of Consular Appointments:\n1\nConsular Appointment Number:\n1\nApplicant Name:\nAjay Kumar\nEmbassy/Consulate/OFC:\nCHENNAI\nStreet Address:\n220 Anna Salai, Opp.Gemini Flyover,\nStreet Address Cont.:\nGemini Circle,', metadata={'source': 'llama2_text.txt'})

In [32]:
text_chunks=text_splitter.split_documents(documents)

print(len(text_chunks))

34


In [33]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device':'cpu'})


#Step 4: Convert the Text Chunks into Embeddings and Create a FAISS Vector Store
vector_store=FAISS.from_documents(text_chunks, embeddings)



In [34]:
vector_store.as_retriever(search_kwargs={'k': 2})

VectorStoreRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x000001D23F9F0F40>, search_type='similarity', search_kwargs={'k': 2})

In [35]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':500,
                          'temperature':0.1})

In [36]:
template="""Use the following pieces of information to answer the user's question.
If you dont know the answer just say you know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else
Helpful answer
"""

qa_prompt=PromptTemplate(template=template, input_variables=['context', 'question'])



chain = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type='stuff',
                                   retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
                                   return_source_documents=True,
                                   chain_type_kwargs={'prompt': qa_prompt})

In [ ]:
question="when was llama announced?"
result=chain({'query':question})
print(result['result'])